## premier test avec pytorch et premier hook

In [3]:
import requests
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch.nn.functional as F
import torch
from causal_tracing import hooks

In [4]:
model_name = "gpt2-large"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token =  tokenizer.eos_token
#tokenizer.padding_side = "left"

In [ ]:
model

In [5]:
# hook pour observer ce qu'il se passe dedans
hook1 = model.transformer.wte.register_forward_hook(hooks.input_hook)
hook2 = model.transformer.wte.register_forward_hook(hooks.output_hook)
hook3 = model.transformer.wpe.register_forward_hook(hooks.input_hook)
hook4 = model.transformer.wpe.register_forward_hook(hooks.output_hook)
hook5 = model.transformer.drop.register_forward_hook(hooks.input_hook)
hook6 = model.transformer.drop.register_forward_hook(hooks.output_hook)


In [6]:
#hook pour ajouter du bruit
noise_hook1 = model.transformer.drop.register_forward_hook(hooks.naive_noise_hook)

In [7]:
# deuxième run avec bruit
input= tokenizer("Audible.com is owned by", return_tensors="pt")
with torch.no_grad():
    outputs = model(**input, labels = input.input_ids, output_hidden_states = True, output_attentions =True)
probs = F.softmax(outputs.logits[0, -1, :], dim=-1)
top_probs, top_indices = torch.topk(probs, 4)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")

Embedding(50257, 1280) : input
(tensor([[16353,   856,    13,   785,   318,  6898,   416]]),)
Embedding(50257, 1280) : output
torch.Size([1, 7, 1280])
tensor([[[-0.0675,  0.0559,  0.0079,  ...,  0.0522,  0.0567, -0.0228],
         [ 0.0624,  0.1533, -0.0481,  ..., -0.0086,  0.0266,  0.0470],
         [ 0.0050,  0.0201, -0.0131,  ...,  0.0461,  0.0038, -0.0206],
         ...,
         [-0.0317, -0.0043, -0.0297,  ...,  0.0258,  0.0306, -0.0274],
         [-0.0416,  0.0511,  0.0218,  ..., -0.0143,  0.0679, -0.0253],
         [-0.0097, -0.0184, -0.0369,  ...,  0.0740, -0.0516, -0.0194]]])
Embedding(1024, 1280) : input
(tensor([[0, 1, 2, 3, 4, 5, 6]]),)
Embedding(1024, 1280) : output
torch.Size([1, 7, 1280])
tensor([[[ 0.0091,  0.0006,  0.0016,  ..., -0.0340, -0.0054, -0.0056],
         [-0.0045,  0.0030, -0.0066,  ...,  0.0045,  0.0127, -0.0031],
         [-0.0008,  0.0031, -0.0019,  ...,  0.0058, -0.0016,  0.0032],
         ...,
         [ 0.0012, -0.0006,  0.0012,  ...,  0.0066, -0.0016

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


.: 0.0280
,: 0.0269
er: 0.0239

: 0.0233


In [10]:
hook1.remove()
hook2.remove()
hook3.remove()
hook4.remove()
hook5.remove()
hook6.remove()
noise_hook1.remove()

## dit si un token se réfère au sujet

In [3]:
url = 'https://rome.baulab.info/data/dsets/known_1000.json'
response = requests.get(url) 
data = response.json()

In [4]:
prompts = [dict['prompt'] for dict in data]
subjects = [dict['subject'] for dict in data]
input= tokenizer(prompts, return_tensors="pt", padding= True, return_offsets_mapping= True)

In [ ]:
mask = []
for j, prompt in enumerate(prompts):
    map = torch.zeros_like(input.input_ids[j], dtype=torch.int)
    for i,t in enumerate(input.offset_mapping[j]):
        
        if (prompts[j].find(subjects[j])-1<=t[0]) and (t[1]<=prompts[j].find(subjects[j])+len(subjects[j])):
            map[i] = 1
    mask.append(map)
masks_tensor = torch.stack(mask)
masks_tensor = torch.logical_and(masks_tensor, input.attention_mask).int()
masks_tensor

pour le prompt i, masks-tensor[i] donne un mask qui dit si oui ou non les tokens se réfèrent au sujet (1 si c'est le cas, 0 sinon)

## rajoute le bruit sur les bons tokens

In [7]:
def noise_hook(module,input,output):
    noise = torch.randn_like(output)#*sqrt(3*variance)
    noisy_output = output + noise * masks_tensor.unsqueeze(-1).float()
    return noisy_output

In [5]:
# fonction qui sert à obtenir le logits du dernier non-padding token

def last_non_padding_token_logits(logits, attention_mask):
    # For each input, find the last non-padding token
    last_non_padding_logits = []
    
    for i in range(logits.size(0)):  # Loop over each prompt in the batch
        # Find the last non-padding token position
        non_padding_positions = (attention_mask[i] == 1).nonzero(as_tuple=True)[0]
        last_non_padding_token_index = non_padding_positions[-1]
        
        # Get the logits of the last non-padding token
        last_non_padding_logits.append(logits[i, last_non_padding_token_index])
    
    return last_non_padding_logits

In [ ]:
#sans le noise
input= tokenizer(prompts, return_tensors="pt", padding= True)
attention_mask = input.attention_mask
with torch.no_grad():
    outputs = model(**input, output_hidden_states = True, output_attentions =True)



In [15]:
#pour le prompt ...
probs = F.softmax(logits_utile[2], dim=-1)
top_probs, top_indices = torch.topk(probs, 5)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")

 CBS: 0.3337
 September: 0.1091
 Tuesday: 0.0595
 Monday: 0.0593
 May: 0.0499


In [ ]:
hook1 = model.transformer.wte.register_forward_hook(input_hook)
masked_noise_hook = model.transformer.drop.register_forward_hook(noise_hook)

input= tokenizer(prompts[:5], return_tensors="pt", padding= True)
with torch.no_grad():
    outputs = model(**input, labels = input.input_ids, output_hidden_states = True, output_attentions =True)

#Le mot prédit pour le 1er prompt avec le noise
probs = F.softmax(outputs.logits[0][-1], dim=-1)
top_probs, top_indices = torch.topk(probs, 4)

# les probas des mots
top_words = [tokenizer.decode([idx]) for idx in top_indices]
for word, prob in zip(top_words, top_probs):
    print(f"{word}: {prob.item():.4f}")